In [ ]:
import os
import torch
import matplotlib.pyplot as plt

In [ ]:
# load model from ./results/model_1.pt
model = torch.load(os.path.join('results', 'model_1.pt'), map_location='cpu')

# evaluate the model on random images of the validation set and print the obtained image

OrderedDict([('encoder.0.weight', tensor([[[[-0.2274,  0.0959, -0.0423],
          [ 0.0619,  0.0118, -0.2014],
          [-0.2305, -0.2653, -0.3366]]],


        [[[ 0.0103,  0.1322,  0.1264],
          [ 0.3434,  0.0582, -0.2602],
          [-0.0146,  0.0835,  0.2482]]],


        [[[-0.0213, -0.1601,  0.1482],
          [ 0.2408, -0.1071, -0.1088],
          [ 0.0876, -0.0800, -0.2487]]],


        [[[-0.0293,  0.1492, -0.3142],
          [-0.2720, -0.0064,  0.2300],
          [ 0.2087, -0.0093, -0.1957]]],


        [[[ 0.3044,  0.3059, -0.1641],
          [-0.1035,  0.3215, -0.0223],
          [-0.2168, -0.0048,  0.0553]]],


        [[[-0.2027, -0.1953,  0.2708],
          [-0.0216,  0.1399, -0.2082],
          [-0.3363, -0.2439, -0.2829]]],


        [[[ 0.2064,  0.1280,  0.2201],
          [ 0.2412, -0.1743, -0.1042],
          [ 0.1609,  0.2351, -0.0888]]],


        [[[ 0.3027,  0.1216,  0.0218],
          [-0.2213,  0.1097, -0.1019],
          [ 0.3076,  0.1707,  0.2027]]],


/tmp/ipykernel_19355/2295568355.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(os.path.join('results', 'model_1.pt'), map_location='cpu')
